In [1]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv('../datasets/sep28k/SEP-28k_labels_new.csv')
print("Total number of samples: ",df.shape)
df.head(5)

Total number of samples:  (21856, 17)


,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,WordRep,DifficultToUnderstand,Interjection,NoStutteredWords,NaturalPause,Music,NoSpeech
0,HeStutters,0,0,31900320,31948320,0,0,0,0,0,0,0,0,3,1,0,0
1,HeStutters,0,1,31977120,32025120,0,0,0,0,0,0,0,0,3,1,0,0
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,0,0,0,3,0,0,0
3,HeStutters,0,3,35200640,35248640,0,0,1,0,0,0,0,0,2,0,0,0
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,0,0,0,3,0,0,0


In [8]:
label_cols = ['Prolongation', 'Block', 'SoundRep', 'WordRep', 'Interjection', 'NoStutteredWords']
df['label_per_type'] = df[label_cols].apply(lambda row: len(label_cols) if row.max() <= 1 else label_cols.index(row.idxmax()), axis=1)
df['label_per_type'].value_counts()


label_per_type
5    9550
4    3311
1    2201
3    2053
0    1800
2    1594
6    1347
Name: count, dtype: int64

In [17]:
# display samples with label_per_type == 4 and NoStutteredWords >= 2
df[(df['label_per_type'] == 4) & (df['NoStutteredWords'] <= 1)]

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,WordRep,DifficultToUnderstand,Interjection,NoStutteredWords,NaturalPause,Music,NoSpeech,label_per_type
64,HeStutters,1,24,30819840,30867840,0,0,0,1,0,1,0,3,1,1,0,0,4
79,HeStutters,1,39,32034720,32082720,0,0,0,1,1,0,0,2,1,1,0,0,4
110,HeStutters,1,70,36630240,36678240,0,0,0,2,2,0,0,3,0,0,0,0,4
149,HeStutters,1,109,6978880,7026880,0,0,0,2,0,0,0,3,1,2,0,0,4
158,HeStutters,1,118,7956000,8004000,0,0,2,0,0,0,0,3,1,1,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21796,WomenWhoStutter,108,20,13630080,13678080,0,0,1,1,0,0,0,2,1,0,0,0,4
21808,WomenWhoStutter,108,32,23310080,23358080,0,0,0,2,1,0,1,3,0,1,0,1,4
21818,WomenWhoStutter,109,2,33036320,33084320,0,0,0,2,0,1,0,3,0,0,0,0,4
21842,WomenWhoStutter,109,26,16538400,16586400,0,0,0,1,0,1,0,3,1,0,0,0,4


In [ ]:
import numpy as np
import statsmodels.stats.inter_rater as irr

# Sample annotation data (replace with your actual data)
# Each row represents a clip and each column represents an annotator's label
# Example:
# 0 - No Dysfluency
# 1 - Word Repetition
# 2 - Interjection
# 3 - Sound Repetition
# 4 - Block
# 5 - Prolongation

annotations = np.array([
    [0, 1, 1],
    [2, 2, 2],
    [0, 0, 0],
    [4, 3, 3],
    [1, 1, 0],
    [5, 5, 5],
    [3, 2, 3],
    [4, 4, 4],
    [0, 1, 1],
    [2, 2, 2]
])

# Convert the annotations to the appropriate format for Fleiss' Kappa
# The format required is a matrix where rows are categories and columns are annotators
categories = np.max(annotations) + 1
table = np.zeros((annotations.shape[0], categories))

for i in range(annotations.shape[0]):
    for j in range(annotations.shape[1]):
        table[i, annotations[i, j]] += 1

# Compute Fleiss' Kappa
fleiss_kappa = irr.fleiss_kappa(table, method='fleiss')

print(f"Fleiss' Kappa: {fleiss_kappa:.2f}")
